
    
# TP 3 Méthodes bayésiennes, MCMC 

#### Adnane EL KASMI - Alexis Aurissergues


# Échantillonneur de Gibbs

Soit $\mu_1$ ~ $\mathcal{N}(\delta_1,\frac{1}{\lambda})$ et $\mu_2$ ~ $\mathcal{N}(\delta_2,\frac{1}{\lambda})$
 

On considère la loi de mélange :
<center>
$\pi(.|\theta)$ = $\sum_{i=1}^2 \pi_{i}\phi(.,\mu_{i},1)$
</center>


Avec $\theta$ = ($\mu_1$, $\mu_2$).
 

On pose $n_1$ = $\sum_{i=1}^{n} \mathbb{1}_{Z_{i} =1} $ et $n_2$ = $ \sum_{i=1}^{n} \mathbb{1}_{Z_{i} =2}$, pour i $\in$ $[1,n]$ ,$Z_{i}$ $\in$ {$1$,$2$}.


<center>
    $\pi(\mu_1|\textbf{X})$ $\propto$ $\pi(\mu_1)$$p_{\mu_1}(\textbf{X})$ 
      $\propto$  $\pi(\mu_1)$${\displaystyle \prod_{i=1}^{n} f_{\mu_1}(X_{i})}$  $\propto$ $e^{\frac{-\lambda(\mu_1 - \delta_1)^2}{2}}$$e^{\sum_{i=1}^{n} \frac{(X_{i}-\mu_1)^2}{2}} \mathbb{1}_{Z_{i} = 1}$ $\propto$ $e^{\frac{-\lambda(\mu_1^2 -2\mu_1\delta_1 + \delta_1)}{2}}$ $e^{-\frac{(\sum_{i=1}^{n_1} X_{i}^2 -2\mu_1X_{i} + \mu_1^2)}{2}}$ $\propto$ $e^{\mu_1^2(-\frac{\lambda}{2} - \frac{n_1}{2}) + \mu_1(\delta_1 +\bar{X}_{n_1}n_1)}$ $\propto$ $e^{( \mu_1 - \frac{ \delta_1 \lambda + \bar{X}_{n_1}}{\lambda + n_1})(\frac{\lambda + n_1}{2})}$

</center>

À une constance multiplicative près, la densité de la loi $\pi (\mu_1 | \textbf{X})$ est la densité d'une loi normale $\mathcal{N}( \frac{\delta_1 \lambda + \bar{X}_{n_1}}{\lambda + n_1}, \frac{1}{\lambda + n_1})$.

De la même manière.


<center>
    $\pi(\mu_2|\textbf{X})$ $\propto$ $\pi(\mu_2)$$p_{\mu_2}(\textbf{X})$ 
      $\propto$  $\pi(\mu_2)$${\displaystyle \prod_{i=1}^{n} f_{\mu_2}(X_{i})}$  $\propto$ $e^{\frac{-\lambda(\mu_2 - \delta_2)^2}{2}}$$e^{\sum_{i=1}^{n} \frac{(X_{i}-\mu_2)^2}{2}} \mathbb{1}_{Z_{i} = 1}$ $\propto$ $e^{\frac{-\lambda(\mu_2^2 -2\mu_2\delta_2 + \delta_2)}{2}}$ $e^{-\frac{(\sum_{i=1}^{n_1} X_{i}^2 -2\mu_2X_{i} + \mu_2^2)}{2}}$ $\propto$ $e^{\mu_2^2(-\frac{\lambda}{2} - \frac{n_1}{2}) + \mu_2(\delta_2 +\bar{X}_{n_1}n_1)}$ $\propto$ $e^{( \mu_2 - \frac{ \delta_2 \lambda + \bar{X}_{n_1}}{\lambda + n_1})(\frac{\lambda + n_1}{2})}$

</center>

À une constance multiplicative près, la densité de la loi $\pi (\mu_2 | \textbf{X})$ est la densité d'une loi normale $\mathcal{N}( \frac{\delta_2 \lambda + \bar{X}_{n_2}}{\lambda + n_2}, \frac{1}{\lambda + n_2})$.




On initialise l'algorithme avec $\theta^{0}$ = ($\mu_1^{0}$,$\mu_2^{0}$).

Pour l'itération t, on note la valeur de $\theta^{(t)}$:

Rappellons que 
$\mathbb{P}( Z_{i}^{(t)} = k| \textbf{X} , \theta^{(t)})$ = $\frac{\pi_{k}^{(t)} \phi(x_i,\mu_{k}^{(t)},1)}{ \sum_{=1}^2  }$

$\pi_k^{(t)} \phi_{x_i},\mu_{k}^{(t)}$

Initialisation : démarrer avec une valeur arbitraire $\mathbf{x}^{(0)}=\left(x_{1}^{(0)}, \ldots, x_{p}^{(0)}\right)$. Iteration $t$ : avec $\left(x_{1}^{(t-1)}, \ldots, x_{p}^{(t-1)}\right)$, generer
1. $x_{1}^{(t)}$ according to $\pi_{1}\left(x_{1} \mid x_{2}^{(t-1)}, \ldots, x_{p}^{(t-1)}\right)$,
2. $x_{2}^{(t)}$ according to $\pi_{2}\left(x_{2} \mid x_{1}^{(t)}, x_{3}^{(t-1)}, \ldots, x_{p}^{(t-1)}\right)$,
3. $x_{p}^{(t)}$ according to $\pi_{p}\left(x_{p} \mid x_{1}^{(t)}, \ldots, x_{p-1}^{(t)}\right)$.

## Implémentation sur R de l’algorithme de l’échantillonneur de Gibbs

In [6]:
Gibbs_Sampling <- function(X,initial,prior,iter) {
    
    

# (Initialisation)
n <- length(X)
mu1 <- initial$mu1
mu2 <- initial$mu2
pi <- initial$pi
delta1 <- prior$delta1
delta2 <- prior$delta2
lambda <- prior$lambda

loglik <- rep(NA, iter)
vec_mu1 <- rep(NA, iter)
vec_mu2 <- rep(NA, iter)

for(t in 1:(iter)){

# (Simulation des variables latentes z_i sachant X et mu)
P <- matrix(NA, length(X), 2)
P[,1]<-pi*dnorm(X,mu1,1)/(pi*dnorm(X,mu1,1)+(1-pi)*dnorm(X,mu2,1))
P[,2]<-(1-pi)*dnorm(X,mu2,1)/(pi*dnorm(X,mu1,1)+(1-pi)*dnorm(X,mu2,1))
Z <- apply(P,1,function(row)sample(1:2, size=1, prob=row))

# (Générer nouveaux candidats)
n1 <- sum(Z==1)
n2 <- sum(Z==2)

s1 <- sum(X[Z==1])
s2 <- sum(X[Z==2])

mu1.new <- rnorm(1, (lambda*delta1+s1)/(lambda+n1), 1/(lambda+n1))
mu2.new <- rnorm(1, (lambda*delta2+s2)/(lambda+n2), 1/(lambda+n2))

# (Mise à jour)
mu1 <- mu1.new
mu2 <- mu2.new

vec_mu1[t] <- mu1
vec_mu2[t] <- mu2

loglik[t] <- logvrais(X,list(pi=c(pi,1-pi), mu=c(mu1,mu2), sigma=c(1,1)))
}

res <- list(mu1=vec_mu1, mu2=vec_mu2, loglik=loglik)
return (res)
}

# Algorithme de Metropolis-Hastings

Comme indiqué dans l'énoncé, on prend comme loi de proposition $Q$ une loi normale qu'on centre en la valeur courante des paramètres, avec $\zeta$ comme écart-type et en simulant des propositions *indépendantes* pour chacun des deux paramètres.

On choisit un nombre d'itérations $T>0$ et on initialise : $(\mu_1^{(0)}, \mu_2^{(0)})$.

Pour $t$ allant de $1$ à $T$ :

1) On simule des variables aléatoires $y_1$ et $y_2$ suivant respectivement les lois $\mathcal{N}(\mu_1^{(t-1)},\zeta^2)$ et  $\mathcal{N}(\mu_2^{(t-1)},\zeta^2)$.

2) On calcule $ r(\mu_k^{(t-1)}, y_k)$ pour $k = 1, 2 $

Où $ r(\mu_k^{(t-1)}, y_k) = \min(\displaystyle \frac{f(y_k) q(\mu_k^{(t-1)} | y_k)}{f(\mu_k^{(t-1)}) q(y_k | \mu_k^{(t-1)})}, 1) $, avec q la densité de la loi de proposition
      
3) Et on pose :

$$
\mu_k^{(t)} = \left\{
    \begin{array}{ll}
        y_k & \mbox{avec probabilité $r( \mu_k^{(t-1)},y_k)$ }   \\
        \mu_k^{(t-1)} & \mbox{avec probabilité  $1-r( \mu_k^{(t-1)},y_k)$}
    \end{array}
\right.
$$

(pour $k = 1, 2$).

## Implémentation sur R de l'algorithme de Metropolis-Hastings

In [10]:
# (on crée une fonction pour la densité du mélange)

f = function(x, m1, m2, p) {
  return(p*dnorm(x, mean = m1, sd = 1)+(1-p)*dnorm(x, mean = m2, sd = 1))
}

Metropolis_Hastings = function(X, initial, prior, zeta, iter) {
  
  # (initialisation)
  n = length(X)
  mu1 = initial$mu1
  mu2 = initial$mu2
  pi = initial$pi
  delta1 = prior$delta1
  delta2 = prior$delta2
  lambda = prior$lambda
  loglik = rep(NA, iter)
  vec_mu1 = rep(NA, iter)
  vec_mu2 = rep(NA, iter)
  
  # (Itérations)
  for(t in 1:iter) {
      
    y1 = rnorm(mean = mu1, sd = zeta)
    y2 = rnorm(mean = mu2, sd = zeta)
    p1 = prod(f(X, mu1, mu2, pi)/f(X, y1, y2, pi))
      
    # (Taux d'acceptation)  
    rho1 = min(1, p*dnorm(x = y1, mean = delta1, sd = 1/lambda)/dnorm(x = mu1, mean = delta1, sd = 1/lambda))
    rho2 = min(1, p*dnorm(x = y2, mean = delta2, sd = 1/lambda)/dnorm(x = mu2, mean = delta2, sd = 1/lambda))
    
    
    u1 = runif(min = 0, max = 1)
    u2 = runif(min = 0, max = 1)
      
    if(u1 < rho1) {
      mu1 = y1
    }
    if(u2 < rho2){
      mu2 = y2
    }
    vec_mu1[t] = mu1
    vec_mu2[t] = mu2
  }
}

# Comparaison des algorithmes

Honnêtement, on n'avait pas beaucoup de temps pour traiter cette partie, mais on a bien compris le pricipe. merci de votre compréhension.